# Pizza pizza pizza

Get top venues by keyword and location, querying Google, Yelp, Foursquare

## 1. Intro
### Motivation
 - Gmaps and Yelp GUIs sorting and filtering are limited.
 - Query all 3 services and bring merged results into sortable table and map widget

### Google Maps setup

 - Needs a Google API key and gmaps python module
 - [Create Google Cloud credentials and give access to Places APIs](https://console.cloud.google.com/google/maps-apis/credentials) (also, restrict IP or set other restrictions)
 - `conda install -c conda-forge -y gmaps`
 - put key in `secrets/apikey.txt`
 - `gmaps` Jupyter nbextension to show maps in notebook, with marker pins etc.

```
conda install -c conda-forge -y jupyter_contrib_nbextensions
jupyter nbextension enable --py gmaps
jupyter notebook
```

### Yelp setup
 - needs Yelp API key and module
 - https://www.yelp.com/developers/documentation/v3
 - https://github.com/gfairchild/yelpapi
 - put key in `secrets/yelpkey.txt`
 
### Foursquare setup
- Needs Foursquare API key and module
- https://developer.foursquare.com/docs/places-api/getting-started/
- https://github.com/mLewisLogic/foursquare
- OAuth id in `secrets/foursquare_id.txt`
- OAuth secret in `secrets/foursquare_secret.txt`

See `requirements.txt` for versions used, other requirements (requests, qgrid, folium, Flask)

Was going to try OpenTable and TripAdvisor but their language seems to limit API key access to approved commercial partners.
- https://dev.opentable.com/affiliate-partners/
- https://www.tripadvisor.com/APIAccessSupport

In [1]:
import time
from pprint import pprint
import ipywidgets
from ipywidgets import widgets, interact
from itertools import product
from os import path

import multiprocessing
from multiprocessing import Pool

import tqdm
from tqdm import tqdm

import traceback
import pdb

import numpy as np
import pandas as pd

# cluster similar entities
import pandas_dedupe

import sklearn
from sklearn.preprocessing import StandardScaler

import requests, json 

import qgrid
from qgrid import show_grid

# convert coords to km using haversine distance 
import geopy
from geopy.distance import distance

import gmaps
with open('secrets/apikey.txt') as f:
    api_key = f.readline().strip()
    f.close
gmaps.configure(api_key=api_key)

# https://github.com/gfairchild/yelpapi
import yelpapi
from yelpapi import YelpAPI
with open('secrets/yelpkey.txt') as f:
    yelp_key = f.readline().strip()
    f.close
yelp_api = YelpAPI(yelp_key)

import foursquare
from foursquare import Foursquare, FoursquareException
with open('secrets/foursquare_id.txt') as f:
    foursquare_id = f.readline().strip()
    f.close
with open('secrets/foursquare_secret.txt') as f:
    foursquare_secret = f.readline().strip()
    f.close

# interactive maps
import folium

print(f"numpy                {np.__version__:<20}")
print(f"pandas               {pd.__version__:<20}")
print(f"ipywidgets           {ipywidgets.__version__:<20}")
print(f"qgrid                {qgrid.__version__:<20}")
print(f"sklearn              {sklearn.__version__:<20}")
print(f"requests             {requests.__version__:<20}")
print(f"geopy                {geopy.__version__:<20}")
print(f"gmaps                {gmaps.__version__:<20}")
print(f"foursquare           {foursquare.__version__:<20}")
print(f"numpy                {folium.__version__:<20}")


numpy                1.18.1              
pandas               1.0.3               
ipywidgets           7.5.1               
qgrid                1.3.1               
sklearn              0.22.1              
requests             2.23.0              
geopy                2.0.0               
gmaps                0.9.0               
foursquare           1!2020.1.30         
numpy                0.11.0              


## 2. Global setup, keyword, location

In [2]:
# pick a search term
keyword_options = [('Pizza', 'pizza'), ('Coffee', 'coffee'), ('Ice Cream', 'icecream')]
keyword = 'pizza'

@interact
def get_kw(kw = widgets.Dropdown(
    options=keyword_options,
    value=keyword,
    description='Search term:  ',
    disabled=False,
)):
    global keyword
    keyword = kw
    return None

interactive(children=(Dropdown(description='Search term:  ', options=(('Pizza', 'pizza'), ('Coffee', 'coffee')…

In [3]:
# pick a location
location = '40.7484, -73.9857'
location_coords = tuple(eval(location))
location_options = [('Midtown', '40.7484, -73.9857'),
                    ('Downtown', '40.7077443,-74.0139089'),
                    ('Upper East Side', '40.7711473,-73.9661166'),
                    ('Upper West Side', '40.778794,-73.984257'),
                    ('Brooklyn Heights', '40.6915812,-73.9954095'), 
                    ('Grand Army Plaza', '40.671872,-73.972544'),
                    ('Bay Ridge', '40.6292633,-74.0309554'),
                    ('Williamsburg', '40.7144609,-73.9553373'),
                  ]

@interact
def get_loc(loc = widgets.Dropdown(
    options=location_options,
    value=location,
    description='Location:',
)):
    global location
    global location_coords
    location = loc
    location_coords = tuple(eval(loc))
    return None

interactive(children=(Dropdown(description='Location:', options=(('Midtown', '40.7484, -73.9857'), ('Downtown'…

In [4]:
test_pickle_filename = keyword + "_" + location.replace(' ', '').lower() + ".pkl"
test_pickle_filename

'pizza_40.7484,-73.9857.pkl'

In [5]:
# pick anything we don't have a pickle file for
for test_keyword, test_location in product(keyword_options, location_options):
    keyword = test_keyword[1]
    location_name = test_location[0]
    location = test_location[1]
    location_coords = tuple(eval(test_location[1]))
    test_pickle_filename = keyword + "_" + location_name.replace(' ', '').lower() + ".pkl"
    
    if not path.exists(test_pickle_filename):
        break
        
print(location_name, keyword, location, location_coords)
print(test_pickle_filename)

Brooklyn Heights pizza 40.6915812,-73.9954095 (40.6915812, -73.9954095)
pizza_brooklynheights.pkl


## 3. Google Maps

In [6]:
# gmap of chosen location
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=location_coords, zoom_level=12, layout=figure_layout)
fig.add_layer(gmaps.marker_layer([location_coords]))
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [7]:
# global options for all search services APIs
MIN_USER_RATINGS = 20
MIN_RATING = 0
NRESULTS = 50
RADIUS = 1000


In [8]:
# gmaps options
GMAPS_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# https://developers.google.com/places/web-service/search#TextSearchRequests
#https://developers.google.com/places/web-service/supported_types
# rankby='prominence'
rankby='distance'
ltype='establishment'


In [9]:
def gmaps_get_first_page(api_key, location, **kwargs):
    """get first page of results from gmaps using api_key, location, kwargs for search spec"""
    # use either rankby or radius kwarg
    request_url = GMAPS_URL + '?key=' + api_key
    request_url += '&location=' + location
    for name, val in kwargs.items():
        request_url += '&' + name + '=' + str(val)
    r = requests.get(request_url)
    j = r.json()
    return j


def gmaps_get_next_page(api_key, next_page_token):
    """get next search engine results page page using search token, waiting until available"""
    r = requests.get(GMAPS_URL + '?pagetoken=' + next_page_token +
                        '&key=' + api_key)
    for i in range(10):
        j = r.json()
        if not j['results']: # wait for next page to be available
            time.sleep(5)
            continue
        else:
            return j


def gmaps_get_all_df(api_key, location, **kwargs):
    """return dataframe of all results using api_key, location, search kwargs"""
    # get first page
    j = gmaps_get_first_page(api_key, location, **kwargs)
    venues_df = pd.json_normalize(j['results'])

    # get pages while additional pages available
    while 'next_page_token' in j:
        next_page_token = j['next_page_token']
        time.sleep(5)
        j = gmaps_get_next_page(api_key, next_page_token)
        venues_df = venues_df.append(pd.json_normalize(j['results']))
        
    return venues_df


def gmaps_get_df(location_coords, keyword):

    # use either rankby or radius
    location = "%.7f,%.7f" % location_coords
    gmaps_df = gmaps_get_all_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
    if gmaps_df.empty:
        return None
    else:
        # gmaps_get_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
        gmaps_df = gmaps_df.loc[(gmaps_df['user_ratings_total'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
                           .sort_values(['rating', 'user_ratings_total'], ascending=False) \
                           .reset_index(drop=True)
        gmaps_df = gmaps_df[['name', 'vicinity', 'rating', 'user_ratings_total', 'geometry.location.lat', 'geometry.location.lng']]
        gmaps_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng']
        # drop trailing ", Brooklyn"
        gmaps_df['address'] = gmaps_df['address'].apply(lambda address: " ".join(address.split(',')[:-1]))
        gmaps_df['distance'] = gmaps_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        gmaps_df['category'] = keyword
        return gmaps_df


In [10]:
%%time
gmaps_df = gmaps_get_df(location_coords, keyword)
gmaps_df = gmaps_df.loc[(gmaps_df['nratings'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
        .sort_values(['rating', 'nratings'], ascending=False) \
        .reset_index(drop=True)
gmaps_df.to_pickle('gmaps_' + test_pickle_filename)
gmaps_df


CPU times: user 68.7 ms, sys: 8.15 ms, total: 76.8 ms
Wall time: 7.26 s


,name,address,rating,nratings,lat,lng,distance,category
0,Juliana's,19 Old Fulton St,4.6,3995,40.702747,-73.993435,1.251088,pizza
1,Sottocasa,298 Atlantic Ave,4.6,683,40.688307,-73.988978,0.654069,pizza
2,Dellarocco's,214 Hicks St,4.6,437,40.695009,-73.996108,0.385215,pizza
3,The House of Pizza & Calzone,132 Union St,4.6,325,40.683997,-74.002281,1.023084,pizza
4,Piz-zetta Pizzeria,90 Livingston St,4.6,238,40.691169,-73.990632,0.406418,pizza
5,Lucali,575 Henry St,4.5,1324,40.681805,-74.000293,1.161421,pizza
6,Table 87,87 Atlantic Ave,4.5,481,40.691282,-73.997321,0.164950,pizza
7,Luzzo's,145 Atlantic Ave,4.5,295,40.690696,-73.995225,0.099529,pizza
8,Brado,155 Atlantic Ave,4.5,248,40.690645,-73.994911,0.112170,pizza
9,Forcella Fried Pizza,445 Albee Square W,4.5,83,40.691106,-73.983176,1.035418,pizza


In [11]:
# sort grid by clicking on header (can also click on filter button)
show_grid(gmaps_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [12]:
# plot on google map

markers = [(row.lat, row.lng) for row in gmaps_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in gmaps_df.itertuples()]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Google rating</dt><dd>{rating}</dd>
<dt>Google reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**row) for i, row in gmaps_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=eval(location), zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [13]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
    
venues_map

## 4. Yelp

In [14]:
def yelp_get_df(location_coords, keyword):
    lat, lng = location_coords
    response = yelp_api.search_query(categories=keyword, latitude=lat, longitude=lng, 
                                     radius=RADIUS, sort_by=rankby, limit=NRESULTS)

    yelp_df = pd.json_normalize(response['businesses'])
    if not yelp_df.empty:    
        yelp_df = yelp_df.loc[(yelp_df['review_count'] >= MIN_USER_RATINGS) & (yelp_df['rating'] >= MIN_RATING)] \
                         .sort_values(['rating', 'review_count'], ascending=False) \
                         .reset_index(drop=True)
        display_columns = ['name', 'location.address1', 'rating', 'review_count', 'coordinates.latitude', 'coordinates.longitude', 'url']
        yelp_df = yelp_df[display_columns]
        yelp_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        yelp_df['distance'] = yelp_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        yelp_df['category'] = keyword
        
        if yelp_df.empty:
            return None
        else:
            return yelp_df
    else:
        return None

yelp_df = yelp_get_df(location_coords, keyword)
yelp_df

,name,address,rating,nratings,lat,lng,url,distance,category
0,Juliana's Pizza,19 Old Fulton St,4.5,2214,40.702615,-73.993416,https://www.yelp.com/biz/julianas-pizza-brookl...,1.236846,pizza
1,Lucali,575 Henry St,4.5,1580,40.681800,-74.000240,https://www.yelp.com/biz/lucali-brooklyn-3?adj...,1.160399,pizza
2,Sottocasa Pizzeria,298 Atlantic Ave,4.5,635,40.688285,-73.989006,https://www.yelp.com/biz/sottocasa-pizzeria-br...,0.653424,pizza
3,Piz-zetta,90 Livingston St,4.5,289,40.691283,-73.990603,https://www.yelp.com/biz/piz-zetta-brooklyn-2?...,0.407626,pizza
4,Front Street Pizza,80 Front St,4.0,358,40.702440,-73.989430,https://www.yelp.com/biz/front-street-pizza-br...,1.307472,pizza
5,La Cigogne,215 Union St,4.0,357,40.683501,-73.999304,https://www.yelp.com/biz/la-cigogne-brooklyn?a...,0.955774,pizza
6,Table 87 - Brooklyn Heights,87 Atlantic Ave,4.0,273,40.691219,-73.997345,https://www.yelp.com/biz/table-87-brooklyn-hei...,0.168461,pizza
7,Dellarocco's,214 Hicks St,4.0,272,40.695031,-73.996125,https://www.yelp.com/biz/dellaroccos-brooklyn-...,0.387829,pizza
8,Enoteca on Court,347 Court St,4.0,254,40.682137,-73.995712,https://www.yelp.com/biz/enoteca-on-court-broo...,1.049122,pizza
9,Fascati Pizza,80 Henry St,4.0,236,40.698500,-73.992730,https://www.yelp.com/biz/fascati-pizza-brookly...,0.801003,pizza


In [15]:
yelp_df.to_pickle('yelp_' + test_pickle_filename)
show_grid(yelp_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [16]:
markers = [(row.lat, row.lng) for row in yelp_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in yelp_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Yelp rating</dt><dd>{rating}</dd>
<dt>Yelp reviews</dt><dd>{nratings}</dd>
</dl>
"""

marker_info = [info_box_template.format(**row) for i, row in yelp_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [17]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## 5. Foursquare

In [18]:
def foursquare_get_df(location_coords, keyword):
    location_str = "%.7f,%.7f" % location_coords
    client = Foursquare(client_id=foursquare_id, 
                                   client_secret=foursquare_secret, 
                                   redirect_uri='http://streeteye.com/')
    response = client.venues.search(params={'query': keyword, 'll': "%s" % location_str, 
                                            'radius': RADIUS, 'limit': NRESULTS})

    foursquare_array = []

    for i, venue in pd.json_normalize(response['venues']).iterrows():
        venue_id = venue['id']
        # query detailed venue info from foursquare
        venue_name = venue['name']
        venue_address = venue['location.address']
        # sometimes no URL
        try:
            venue_url = venue['delivery.url']
        except:
            venue_url = ''
        venue_lat = venue['location.lat']
        venue_lng = venue['location.lng']
        # default these to -1
        try:
            # get rating, nratings with another API call for venue details
            venue_details = client.venues(venue_id)['venue']
            venue_rating = venue_details['rating']
            venue_nratings = venue_details['ratingSignals']
        except FoursquareException as e:
            print("Foursquare exception", type(e), str(e))
        except Exception as e:
            continue
            # sometimes no rating ... probably not popular enough
            # print(type(e), str(e))
            # print(traceback.format_exc())
            # print("No rating for %s" % venue_name)

        foursquare_array.append([venue_name, venue_address, venue_rating, venue_nratings, venue_lat, venue_lng, venue_url])
            
    foursquare_df = pd.DataFrame(foursquare_array)
    
    if foursquare_df.empty:
        return None
    else:
        foursquare_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        foursquare_df = foursquare_df.loc[(foursquare_df['nratings'] >= MIN_USER_RATINGS) & (foursquare_df['rating'] >= MIN_RATING)] \
                                     .sort_values(['rating', 'nratings'], ascending=False) \
                                     .reset_index(drop=True)
        
        foursquare_df['distance'] = foursquare_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                                        axis=1)
        foursquare_df['category'] = keyword
        
        return foursquare_df

foursquare_df = foursquare_get_df(location_coords, keyword)
foursquare_df

,name,address,rating,nratings,lat,lng,url,distance,category
0,Lucali,575 Henry St,9.3,839,40.681822,-74.000352,NaN,1.161511,pizza
1,Dellarocco's,214 Hicks St,9.0,244,40.694964,-73.996109,https://www.seamless.com/menu/dellaroccos-pizz...,0.380278,pizza
2,My Little Pizzeria,114 Court St,8.1,163,40.690236,-73.992334,NaN,0.299848,pizza
3,Fatoosh BBQ & Pizza,330 Hicks St,8.1,63,40.691328,-73.997739,NaN,0.198944,pizza
4,Brooklyn Pizza Market,267 Smith St,7.9,54,40.682696,-73.993191,https://www.seamless.com/menu/brooklyn-pizza-m...,1.004384,pizza
5,Numero 28 Pizza & Cucina,68 Bergen St,7.8,117,40.686871,-73.990721,NaN,0.656206,pizza
6,Fascati Pizza,80 Henry St,7.8,109,40.698355,-73.992522,NaN,0.790874,pizza
7,Layla Jones,214 Court St,7.3,76,40.686613,-73.993821,https://www.seamless.com/menu/layla-jones-214-...,0.567762,pizza
8,Sal's Pizzeria,305 Court St,7.2,134,40.683752,-73.995208,NaN,0.869614,pizza
9,Enoteca on Court,347 Court St,7.2,99,40.682215,-73.995947,https://www.seamless.com/menu/enoteca-on-court...,1.041140,pizza


In [19]:
if foursquare_df is not None and not foursquare_df.empty:
    foursquare_df.to_pickle('foursquare_' + test_pickle_filename)
    
show_grid(foursquare_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [20]:
markers = [(row.lat, row.lng) for row in foursquare_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in foursquare_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Foursquare rating</dt><dd>{rating}</dd>
<dt>Foursquare reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**d_item) for i, d_item in foursquare_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [21]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## 6. Merge dataframes after deduplication

In [22]:
# put everything in one big df
pd.set_option('display.max_rows', None)

gmaps_df_copy, yelp_df_copy, foursquare_df_copy = None, None, None

try:
    gmaps_df_copy = gmaps_df.copy()
    gmaps_df_copy['source'] = '0_gmaps'
except: 
    pass

try:
    yelp_df_copy = yelp_df.copy() 
    yelp_df_copy['source'] = '1_yelp'
except:
    pass

try:
    foursquare_df_copy = foursquare_df.copy()
    foursquare_df_copy['source'] = '2_foursquare'
except:
    pass


venues_df = pd.concat(list(filter(lambda df: df is not None, [gmaps_df_copy, yelp_df_copy, foursquare_df_copy]))).reset_index()
venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)

venues_df.sort_values('name')

,index,name,address,rating,nratings,lat,lng,distance,category,source,url,latlong
21,21,2 Bros. Pizza,395 Flatbush Ave Ext #5321,4.1,457,40.689439,-73.980940,1.245994,pizza,0_gmaps,NaN,"(40.6894392, -73.9809399)"
63,28,99 Cent Fresh Hot Pizza,51 Willoughby St,3.5,26,40.692261,-73.986886,0.724431,pizza,1_yelp,https://www.yelp.com/biz/99-cent-fresh-hot-piz...,"(40.6922610104084, -73.9868856221437)"
84,15,99 Cent Pizza,51D Willoughby St,5.3,21,40.692165,-73.986972,0.716142,pizza,2_foursquare,https://www.seamless.com/menu/99-cent-supreme-...,"(40.692165329405405, -73.98697185450945)"
23,23,99 Cents Hot Pizza,255 Livingston St,4.1,121,40.688824,-73.983285,1.069644,pizza,0_gmaps,NaN,"(40.68882420000001, -73.98328459999999)"
25,25,99 cent supreme pizza,44 Willoughby St,4.1,20,40.692140,-73.987156,0.700372,pizza,0_gmaps,NaN,"(40.6921401, -73.9871562)"
28,28,Antonio's Pizzeria,32 Court St,3.9,109,40.692952,-73.991008,0.401936,pizza,0_gmaps,NaN,"(40.6929516, -73.99100829999999)"
61,26,Antonio's Pizzeria & Cafe,32 Court St,3.5,72,40.692966,-73.991046,0.399573,pizza,1_yelp,https://www.yelp.com/biz/antonios-pizzeria-and...,"(40.6929657361683, -73.9910462092274)"
33,33,Bella Pizza,208 Livingston St #5811,3.0,38,40.689476,-73.985988,0.829980,pizza,0_gmaps,NaN,"(40.689476, -73.98598799999999)"
50,15,Brado,155 Atlantic Ave,4.0,138,40.690557,-73.995479,0.113885,pizza,1_yelp,https://www.yelp.com/biz/brado-brooklyn-3?adju...,"(40.6905570310882, -73.9954792872909)"
8,8,Brado,155 Atlantic Ave,4.5,248,40.690645,-73.994911,0.112170,pizza,0_gmaps,NaN,"(40.69064520000001, -73.9949106)"


In [23]:
# manual fix 
# venues_df.loc[venues_df['address']=='212b Pacific St', 'address'] = '212 Pacific St'
# venues_df

In [24]:
# run dedupe algorithm using name, address as default texts, latlong as latlong
venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])
venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['category', 'name', 'address', ('latlong', 'LatLong')])


Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 48


In [25]:
# view clustering
venues_df['cluster'] = venues_df2['cluster id']
clustercols = ['cluster', 'name', 'address', 'rating', 'category', 'nratings', 'lat', 'lng', 'distance', 'source']
venues_df = venues_df.sort_values(['cluster', 'source'])[clustercols]
venues_df

,cluster,name,address,rating,category,nratings,lat,lng,distance,source
0,0,Juliana's,19 Old Fulton St,4.6,pizza,3995,40.7027467,-73.9934349,1.2510884703746683,0_gmaps
35,0,Juliana's Pizza,19 Old Fulton St,4.5,pizza,2214,40.7026153030093,-73.9934159993549,1.2368456393824396,1_yelp
1,1,Sottocasa,298 Atlantic Ave,4.6,pizza,683,40.6883065,-73.9889778,0.6540693037982467,0_gmaps
37,1,Sottocasa Pizzeria,298 Atlantic Ave,4.5,pizza,635,40.688285,-73.989006,0.6534238545711961,1_yelp
2,2,Dellarocco's,214 Hicks St,4.6,pizza,437,40.6950091,-73.9961082,0.38521527537194344,0_gmaps
42,2,Dellarocco's,214 Hicks St,4.0,pizza,272,40.6950309,-73.9961252,0.38782921088062255,1_yelp
70,2,Dellarocco's,214 Hicks St,9.0,pizza,244,40.694964,-73.996109,0.38027750154159473,2_foursquare
3,3,The House of Pizza & Calzone,132 Union St,4.6,pizza,325,40.6839972,-74.00228129999999,1.023083746411481,0_gmaps
49,3,House of Pizza & Calzones,132 Union St,4.0,pizza,145,40.683944,-74.00225,1.0264597243622748,1_yelp
79,3,House of Pizza & Calzones,132 Union St,7.2,pizza,48,40.68403941172705,-74.00215525440561,1.0131938845118804,2_foursquare


In [26]:
# group by clusters
cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'distance', 'source', 'category']] \
    .first() \
    .reset_index()
cluster_df

,cluster,name,address,lat,lng,distance,source,category
0,0,Juliana's,19 Old Fulton St,40.7027467,-73.9934349,1.2510884703746683,0_gmaps,pizza
1,1,Sottocasa,298 Atlantic Ave,40.6883065,-73.9889778,0.6540693037982467,0_gmaps,pizza
2,2,Dellarocco's,214 Hicks St,40.6950091,-73.9961082,0.38521527537194344,0_gmaps,pizza
3,3,The House of Pizza & Calzone,132 Union St,40.6839972,-74.00228129999999,1.023083746411481,0_gmaps,pizza
4,4,Piz-zetta Pizzeria,90 Livingston St,40.6911691,-73.9906319,0.4064180424981932,0_gmaps,pizza
5,5,Lucali,575 Henry St,40.6818053,-74.0002926,1.1614210082934948,0_gmaps,pizza
6,6,Table 87,87 Atlantic Ave,40.6912825,-73.9973211,0.16495006034714688,0_gmaps,pizza
7,7,Luzzo's,145 Atlantic Ave,40.690696,-73.99522499999999,0.09952903372102893,0_gmaps,pizza
8,8,Front Street Pizza,80 Front St,40.7023963,-73.9894556,1.302160081002831,0_gmaps,pizza
9,9,My Little Pizzeria,114 Court St,40.6902356,-73.992334,0.2998479748841386,0_gmaps,pizza


In [27]:
# make markers on clusters
# add ratings on all rows

markers = [(float(row.lat), float(row.lng)) for row in cluster_df.itertuples()]
marker_hover = ["%s" % (row.name) for row in cluster_df.itertuples()]

# make a dict by cluster, initialize rating string to ''
marker_dict = {i: {'name': row['name'],
                   'address': row['address'],
                   'lat': row['lat'],
                   'lng': row['lng'],
                   'rate_str': '',
                  } for i, row in cluster_df.iterrows()}

# add all rating strings
for i, row in venues_df.iterrows():
    cluster = row['cluster']
    if row.source[2:] == 'gmaps':
        marker_dict[cluster]['rate_str'] += "<dt>Google rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'yelp':
        marker_dict[cluster]['rate_str'] += "<dt>Yelp rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'foursquare':
        marker_dict[cluster]['rate_str'] += "<dt>Foursquare rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
{rate_str}

</dl>
"""    

marker_info = [info_box_template.format(**d_item) for i, d_item in marker_dict.items()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [28]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map


In [29]:
merge_df = cluster_df \
    .merge(venues_df.loc[venues_df['source']=='0_gmaps'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'gmaps_rating', 'nratings': 'gmaps_nratings'})
merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='1_yelp'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'yelp_rating', 'nratings': 'yelp_nratings'})
merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='2_foursquare'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'foursquare_rating', 'nratings': 'foursquare_nratings'})
merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

merge_df 

,cluster,name,address,lat,lng,distance,source,category,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std
0,0,Juliana's,19 Old Fulton St,40.7027467,-73.9934349,1.2510884703746683,0_gmaps,pizza,4.6,3995,1.045520,4.5,2214,1.337296,NaN,NaN,NaN
1,1,Sottocasa,298 Atlantic Ave,40.6883065,-73.9889778,0.6540693037982467,0_gmaps,pizza,4.6,683,1.045520,4.5,635,1.337296,NaN,NaN,NaN
2,2,Dellarocco's,214 Hicks St,40.6950091,-73.9961082,0.38521527537194344,0_gmaps,pizza,4.6,437,1.045520,4.0,272,0.462910,9.0,244,1.547012
3,3,The House of Pizza & Calzone,132 Union St,40.6839972,-74.00228129999999,1.023083746411481,0_gmaps,pizza,4.6,325,1.045520,4.0,145,0.462910,7.2,48,-0.134269
4,4,Piz-zetta Pizzeria,90 Livingston St,40.6911691,-73.9906319,0.4064180424981932,0_gmaps,pizza,4.6,238,1.045520,4.5,289,1.337296,NaN,NaN,NaN
5,5,Lucali,575 Henry St,40.6818053,-74.0002926,1.1614210082934948,0_gmaps,pizza,4.5,1324,0.804775,4.5,1580,1.337296,9.3,839,1.827225
6,6,Table 87,87 Atlantic Ave,40.6912825,-73.9973211,0.16495006034714688,0_gmaps,pizza,4.5,481,0.804775,4.0,273,0.462910,NaN,NaN,NaN
7,7,Luzzo's,145 Atlantic Ave,40.690696,-73.99522499999999,0.09952903372102893,0_gmaps,pizza,4.5,295,0.804775,4.0,182,0.462910,NaN,NaN,NaN
8,8,Front Street Pizza,80 Front St,40.7023963,-73.9894556,1.302160081002831,0_gmaps,pizza,4.4,1174,0.564030,4.0,358,0.462910,NaN,NaN,NaN
9,9,My Little Pizzeria,114 Court St,40.6902356,-73.992334,0.2998479748841386,0_gmaps,pizza,4.4,548,0.564030,3.5,260,-0.411476,8.1,163,0.706371


In [30]:
# simple average score
merge_df['meanrating'] = np.nanmean(merge_df[['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']], axis=1)
merge_df.sort_values('meanrating', ascending=False)[['name', 'address', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'meanrating']]


,name,address,gmaps_rating,yelp_rating,foursquare_rating,meanrating
5,Lucali,575 Henry St,4.5,4.5,9.3,1.323099
0,Juliana's,19 Old Fulton St,4.6,4.5,NaN,1.191408
1,Sottocasa,298 Atlantic Ave,4.6,4.5,NaN,1.191408
4,Piz-zetta Pizzeria,90 Livingston St,4.6,4.5,NaN,1.191408
2,Dellarocco's,214 Hicks St,4.6,4.0,9.0,1.018480
27,Forcella Fried Pizza,445 Albee Square W,4.5,NaN,NaN,0.804775
26,Brado,155 Atlantic Ave,4.5,NaN,NaN,0.804775
6,Table 87,87 Atlantic Ave,4.5,4.0,NaN,0.633843
7,Luzzo's,145 Atlantic Ave,4.5,4.0,NaN,0.633843
24,Fatoosh Pitza & Grill,330 Hicks St,NaN,4.0,8.1,0.584641


## 7. Bayesian estimated score

In [31]:
# bayes score
rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
nratings_mean = np.mean(merge_df['nratings'])
rating_avg = np.nanmean(merge_df[rating_cols])
merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
print('mean number of ratings', nratings_mean)
print('average rating', rating_avg)
merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg

show_grid(merge_df.sort_values('bayes_score', ascending=False)[['name', 'address', 'category', 'distance', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'nratings', 'bayes_score']])

mean number of ratings 1.7708333333333333
average rating -7.314410515177502e-16


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [32]:
def dedupe(dedupe_list):

    for i, source_df in enumerate(dedupe_list):
        source_df['source'] = i
    venues_df = pd.concat(dedupe_list).reset_index()
    venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)
    venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])

    # dedupe and assign cluster id
    venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['category', 'shortname', 'address', ('latlong', 'LatLong')])
    venues_df['cluster'] = venues_df2['cluster id']
    venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'category', 'rating', 'nratings', 'lat', 'lng', 'distance', 'source']]

    # group by clusters, uniquify name
    cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'distance', 'source', 'category']] \
                          .first() \
                          .reset_index()

    # merge ratings by source
    merge_df = cluster_df \
        .merge(venues_df.loc[venues_df['source']=='0'][['cluster','rating', 'nratings', 'category']], on='cluster', how='outer') \
        .rename(columns={'rating': 'gmaps_rating', 'nratings': 'gmaps_nratings'})
    merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='1'][['cluster','rating', 'nratings', 'category']], on='cluster', how='outer') \
        .rename(columns={'rating': 'yelp_rating', 'nratings': 'yelp_nratings'})
    merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='2'][['cluster','rating', 'nratings', 'category']], on='cluster', how='outer') \
        .rename(columns={'rating': 'foursquare_rating', 'nratings': 'foursquare_nratings'})
    merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

    # bayes score
    rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
    merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
    nratings_mean = np.mean(merge_df['nratings'])
    rating_avg = np.nanmean(merge_df[rating_cols])
    merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
    merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
    merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg
    merge_df = merge_df.sort_values('bayes_score', ascending=False)
    return merge_df

    
dedupe_list = list(filter(lambda df: df is not None, [gmaps_df, yelp_df, foursquare_df]))

dedupe_df = dedupe(dedupe_list)
dedupe_df


Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 48


,cluster,name,address,lat,lng,distance,source,category_x,gmaps_rating,gmaps_nratings,...,category_x,yelp_rating_std,foursquare_rating,foursquare_nratings,category_y,foursquare_rating_std,nratings,w,R,bayes_score
5,5,Lucali,575 Henry St,40.6818053,-74.0002926,1.1614210082934948,0,pizza,4.5,1324,...,pizza,1.337296,9.3,839,pizza,1.827225,3,0.628821,1.323099,0.831992
2,2,Dellarocco's,214 Hicks St,40.6950091,-73.9961082,0.38521527537194344,0,pizza,4.6,437,...,pizza,0.462910,9.0,244,pizza,1.547012,3,0.628821,1.018480,0.640442
0,0,Juliana's,19 Old Fulton St,40.7027467,-73.9934349,1.2510884703746683,0,pizza,4.6,3995,...,pizza,1.337296,NaN,NaN,NaN,NaN,2,0.530387,1.191408,0.631907
1,1,Sottocasa,298 Atlantic Ave,40.6883065,-73.9889778,0.6540693037982467,0,pizza,4.6,683,...,pizza,1.337296,NaN,NaN,NaN,NaN,2,0.530387,1.191408,0.631907
4,4,Piz-zetta Pizzeria,90 Livingston St,40.6911691,-73.9906319,0.4064180424981932,0,pizza,4.6,238,...,pizza,1.337296,NaN,NaN,NaN,NaN,2,0.530387,1.191408,0.631907
7,7,Luzzo's,145 Atlantic Ave,40.690696,-73.99522499999999,0.09952903372102893,0,pizza,4.5,295,...,pizza,0.462910,NaN,NaN,NaN,NaN,2,0.530387,0.633843,0.336182
6,6,Table 87,87 Atlantic Ave,40.6912825,-73.9973211,0.16495006034714688,0,pizza,4.5,481,...,pizza,0.462910,NaN,NaN,NaN,NaN,2,0.530387,0.633843,0.336182
24,24,Fatoosh Pitza & Grill,330 Hicks St,40.6915,-73.99777,0.199727742106794,1,pizza,NaN,NaN,...,pizza,0.462910,8.1,63,pizza,0.706371,2,0.530387,0.584641,0.310086
27,27,Forcella Fried Pizza,445 Albee Square W,40.6911058,-73.9831758,1.0354179159576993,0,pizza,4.5,83,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.360902,0.804775,0.290445
26,26,Brado,155 Atlantic Ave,40.69064520000001,-73.9949106,0.11216972425363314,0,pizza,4.5,248,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.360902,0.804775,0.290445


In [33]:
dedupe_df.to_pickle(test_pickle_filename)
pd.read_pickle(test_pickle_filename)

,cluster,name,address,lat,lng,distance,source,category_x,gmaps_rating,gmaps_nratings,...,category_x,yelp_rating_std,foursquare_rating,foursquare_nratings,category_y,foursquare_rating_std,nratings,w,R,bayes_score
5,5,Lucali,575 Henry St,40.6818053,-74.0002926,1.1614210082934948,0,pizza,4.5,1324,...,pizza,1.337296,9.3,839,pizza,1.827225,3,0.628821,1.323099,0.831992
2,2,Dellarocco's,214 Hicks St,40.6950091,-73.9961082,0.38521527537194344,0,pizza,4.6,437,...,pizza,0.462910,9.0,244,pizza,1.547012,3,0.628821,1.018480,0.640442
0,0,Juliana's,19 Old Fulton St,40.7027467,-73.9934349,1.2510884703746683,0,pizza,4.6,3995,...,pizza,1.337296,NaN,NaN,NaN,NaN,2,0.530387,1.191408,0.631907
1,1,Sottocasa,298 Atlantic Ave,40.6883065,-73.9889778,0.6540693037982467,0,pizza,4.6,683,...,pizza,1.337296,NaN,NaN,NaN,NaN,2,0.530387,1.191408,0.631907
4,4,Piz-zetta Pizzeria,90 Livingston St,40.6911691,-73.9906319,0.4064180424981932,0,pizza,4.6,238,...,pizza,1.337296,NaN,NaN,NaN,NaN,2,0.530387,1.191408,0.631907
7,7,Luzzo's,145 Atlantic Ave,40.690696,-73.99522499999999,0.09952903372102893,0,pizza,4.5,295,...,pizza,0.462910,NaN,NaN,NaN,NaN,2,0.530387,0.633843,0.336182
6,6,Table 87,87 Atlantic Ave,40.6912825,-73.9973211,0.16495006034714688,0,pizza,4.5,481,...,pizza,0.462910,NaN,NaN,NaN,NaN,2,0.530387,0.633843,0.336182
24,24,Fatoosh Pitza & Grill,330 Hicks St,40.6915,-73.99777,0.199727742106794,1,pizza,NaN,NaN,...,pizza,0.462910,8.1,63,pizza,0.706371,2,0.530387,0.584641,0.310086
27,27,Forcella Fried Pizza,445 Albee Square W,40.6911058,-73.9831758,1.0354179159576993,0,pizza,4.5,83,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.360902,0.804775,0.290445
26,26,Brado,155 Atlantic Ave,40.69064520000001,-73.9949106,0.11216972425363314,0,pizza,4.5,248,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.360902,0.804775,0.290445


In [34]:
def callback(*args):
    print(args)
    
callback(1,3,4)

(1, 3, 4)


In [35]:
%%time
# run all in parallel

services = ['gmaps', 'yelp', 'foursquare']
args = list(product(services, [location_coords], [keyword]))

def generic_get_df(argslist):
    """Query specified service for location and keyword, return dataframe"""
    
    service, location_coords, keyword = argslist
    
    if service=="gmaps":
        retdf = gmaps_get_df(location_coords, keyword)
    elif service=="yelp":
        retdf = yelp_get_df(location_coords, keyword)
    elif service=="foursquare":
        retdf = foursquare_get_df(location_coords, keyword)
        
    return retdf

def all_get_df(location_coords, keyword):
    """Query all services for location and keyword, return list of dataframes"""

    services = ['gmaps', 'yelp', 'foursquare']
    args = list(product(services, [location_coords], [keyword]))
    with Pool() as pool:
        # pool.map will block till all return
        # pool.imap_unordered should make each available as returned … seems to not return, incompatible with underlying apis I guess
        df_list = pool.map(generic_get_df, args)
    return df_list

print("Querying…")
df_list = all_get_df(location_coords, keyword) 
print("Deduping…")
dedupe_df =  dedupe([df for df in df_list if df is not None])
dedupe_df

Querying…
Deduping…
Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 48
CPU times: user 167 ms, sys: 239 ms, total: 407 ms
Wall time: 7.96 s


,cluster,name,address,lat,lng,distance,source,category_x,gmaps_rating,gmaps_nratings,...,category_x,yelp_rating_std,foursquare_rating,foursquare_nratings,category_y,foursquare_rating_std,nratings,w,R,bayes_score
5,5,Lucali,575 Henry St,40.6818053,-74.0002926,1.1614210082934948,0,pizza,4.5,1324,...,pizza,1.337296,9.3,839,pizza,1.827225,3,0.628821,1.323099,0.831992
2,2,Dellarocco's,214 Hicks St,40.6950091,-73.9961082,0.38521527537194344,0,pizza,4.6,437,...,pizza,0.462910,9.0,244,pizza,1.547012,3,0.628821,1.018480,0.640442
0,0,Juliana's,19 Old Fulton St,40.7027467,-73.9934349,1.2510884703746683,0,pizza,4.6,3995,...,pizza,1.337296,NaN,NaN,NaN,NaN,2,0.530387,1.191408,0.631907
1,1,Sottocasa,298 Atlantic Ave,40.6883065,-73.9889778,0.6540693037982467,0,pizza,4.6,683,...,pizza,1.337296,NaN,NaN,NaN,NaN,2,0.530387,1.191408,0.631907
4,4,Piz-zetta Pizzeria,90 Livingston St,40.6911691,-73.9906319,0.4064180424981932,0,pizza,4.6,238,...,pizza,1.337296,NaN,NaN,NaN,NaN,2,0.530387,1.191408,0.631907
7,7,Luzzo's,145 Atlantic Ave,40.690696,-73.99522499999999,0.09952903372102893,0,pizza,4.5,295,...,pizza,0.462910,NaN,NaN,NaN,NaN,2,0.530387,0.633843,0.336182
6,6,Table 87,87 Atlantic Ave,40.6912825,-73.9973211,0.16495006034714688,0,pizza,4.5,481,...,pizza,0.462910,NaN,NaN,NaN,NaN,2,0.530387,0.633843,0.336182
24,24,Fatoosh Pitza & Grill,330 Hicks St,40.6915,-73.99777,0.199727742106794,1,pizza,NaN,NaN,...,pizza,0.462910,8.1,63,pizza,0.706371,2,0.530387,0.584641,0.310086
27,27,Forcella Fried Pizza,445 Albee Square W,40.6911058,-73.9831758,1.0354179159576993,0,pizza,4.5,83,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.360902,0.804775,0.290445
26,26,Brado,155 Atlantic Ave,40.69064520000001,-73.9949106,0.11216972425363314,0,pizza,4.5,248,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0.360902,0.804775,0.290445


In [36]:
# # save a file to retrain with train_dedupe2.py
# tempdf = None
# for k, l, j in product(['pizza', 'coffee', 'icecream'], 
#                     ['midtown','downtown','uppereastside','upperwestside','brooklynheights','grandarmyplaza','bayridge','williamsburg',],
#                     ['gmaps', 'yelp', 'foursquare']):
#     filename = "cache/%s_%s_%s.pkl" % (j, k, l)
#     try:
#         if tempdf is None:
#             tempdf = pd.read_pickle(filename)
#             tempdf['keyword']=k
#             print(filename)
#         else:
#             tempdf2 = pd.read_pickle(filename)
#             tempdf2['keyword']=k
#             tempdf = pd.concat([tempdf, tempdf2])
#             print(filename)
#     except:
#         print("missing ", filename)
# tempdf.to_csv('train_df.csv', index=False)

In [38]:
# wanted to do a tqdm progress bar using imap_unordered

def _foo(my_number):
    square = my_number * my_number
    time.sleep(1)
    return square 

NSTEPS=30

with Pool(multiprocessing.cpu_count()) as p:
    r = list(tqdm(p.imap_unordered(_foo, range(NSTEPS)), total=NSTEPS))

100%|██████████| 30/30 [00:02<00:00, 14.99it/s]
